In [1]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore
from rdflib import Graph, Namespace, URIRef

from xml.dom.minidom import parse
import xml.etree.ElementTree as etree
from Marc_to_Bibframe.Marc.marcWork import MarcWork
from Marc_to_Bibframe.Marc.marcInstance import MarcInstance
from Marc_to_Bibframe.Marc.marcItems import MarcItems
from Marc_to_Bibframe.Work.work import Work
from Marc_to_Bibframe.Instance.instance import Instance

import pysolr
from Solr.solr import create_doc

In [13]:
#JENA
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/bibframe/update')
query_endpoint = 'http://localhost:3030/bibframe/query'
update_endpoint = 'http://localhost:3030/bibframe/update'
store.open((query_endpoint, update_endpoint))

In [6]:
#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":18,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-1"}},\n  "status":"OK"}\n'

In [15]:
def Make_Graph(nt, bf, count ):
    G1 = "PREFIX bk: <http://bibliokeia.com/bibframe/"+bf+"""/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {
        GRAPH bk:"""

    G2 = " {"

    G3 = """}
    }"""

    G = G1+str(count)+" { \n"+nt+"}}"

    return G

In [ ]:
def Upadate(path_marc, count, shelf, out):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(records.index(record))
        print(workMarc.Title().get('title'))

        #SOLR
        doc = create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)

        # #JENA
        # BFwork = URIRef(f"http://bibliokeia.com/bibframe/work/{count}") 
        # BFinstance = URIRef(f"http://bibliokeia.com/bibframe/instance/{count}")
        # #Work
        # work = Work(count, workMarc, BFwork, BFinstance)
        # work.serialize(f'out/{out}/works/{count}.ttl', format='turtle')
        # nt = work.serialize(format='nt')
        # W = Make_Graph(nt, "work", count)
        # store.update(W)
        # #Instance
        # instance = Instance(count, workMarc, instanceMarc, itemsMarc, BFwork, BFinstance, shelf)
        # i = instance.get('instance')
        # i.serialize(f'out/{out}/instances/{count}.ttl', format='turtle')
        # i_nt = i.serialize(format='nt')
        # I = Make_Graph(i_nt, "instance", count)
        # store.update(I)
        # #Items
        # items = instance.get('items')
        # for item, register in items:
        #     item.serialize(f'out/{out}/items/{register}.ttl', format='turtle')
        #     item_nt = item.serialize(format='nt')
        #     Item = Make_Graph(item_nt, "item", register)
        #     store.update(Item)

        count += 1
    solr.add(docs, commit=True)

E = 'E1'
P = 'P1'

out = 'E2/P1'
path_marc = f'Input/{out}/koha.xml'
count = 105
Upadate(path_marc, count, "E2.P1", out)

In [4]:
def UpadateSolr(path_marc, count, shelf, out):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(records.index(record))
        print(workMarc.Title().get('title'))

        #SOLR
        doc = create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)

        count += 1

    return docs

In [8]:
count = 1
e = '1'
p = '1'
out = f'E{e}/P{p}'
path_marc = f'Input/{out}/koha.xml'

docs = UpadateSolr(path_marc, count, f"E{e}.P{p}", out)
solr.add(docs, commit=True)

0
Conjecturas e refutações
1
Os donos da paisagem
2
Cooperação Brasil-França
3
<I=     1> Bienal de Pesquisa da Fundação Oswaldo Cruz, 7 a 11 de dezembro de 1998.
4
Contribuição da pós-graduação brasileira para o desenvolvimento sustentável
5
FAPESP
6
Para uma história da FAPESP
7
Relatório de atividades 2003-2008
8
V Jornada científica de pos-graduação da FIOCRUZ
9
Anais
10
Anais
11
Relatório de atividades 2007
12
Resumos
13
Resumos
14
Resumos
15
Resumos
16
Resumos
17
Relatório de atividades 2010


'{\n  "responseHeader":{\n    "status":0,\n    "QTime":1244}}\n'

In [7]:
docs[0]

{'id': 1,
 'call': '001 P831c',
 'shelf': 'E1.P1',
 'title': 'Conjecturas e refutações',
 'responsibilities': 'Karl R. Popper; tradução de Sérgio Bath.',
 'place': 'Brasília',
 'publisher': 'Ed. Universidade de Brasília',
 'subject': ['Ciência--Metodologia', 'Teoria do conhecimento'],
 'type': 'Livro',
 'items': ['01-0519'],
 'author': 'Popper, Karl Raimund',
 'serie': 'Pensamento científico ; n. 1',
 'year': '1982'}

In [26]:
workMarc.Language()

' po'

In [25]:
print(workMarc.Title().get('title'))

Dominando o PostgreSQL


In [4]:
g = Graph()
g.parse('cli/Out/works/14649397.bibframe.rdf')

<Graph identifier=Ne84daf5ad74a41efbade85675acd9138 (<class 'rdflib.graph.Graph'>)>